In [31]:
import pandas as pd
import numpy as np
from torch_geometric.data import Data, DataLoader
from torch.optim import Adam
from pathlib import Path
from tqdm.auto import tqdm
import torch.nn as nn
import torch_geometric.nn as gnn
import torch.nn.functional as F

In [2]:
MAP2D_FOLDER = '../data/nsp_distances_angles2/'
TARGET_COLS = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']
NEPTUNE_API_TOKEN='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiODQwOTM5MjItYWQ2Mi00ODRhLTgxOTUtMzA4NzNhMzI3OGIwIn0='

In [3]:
import torch

In [4]:
import pytorch_lightning as pl

In [5]:
adjency = np.load('../data/nsp_distances_angles2/id_00073f8be.npy')

In [6]:
def knn_from_adjency(adjency, k=2):
    edges = {(i,j) for i,arr in enumerate(np.argpartition(adjency,k)[:,:k]) for j in arr if i!=j}
    return sorted(edges)

In [7]:
df =  pd.read_json('../data/train.json', lines=True)

In [8]:
def seq2nodes(sequence):
    type_dict={'A':0,'G':1,'U':2,'C':3}
    nodes=np.zeros((len(sequence),4))
    for i,s in enumerate(sequence):
        nodes[i,type_dict[s]]=1
    return nodes

In [9]:
edges_index = np.array(knn_from_adjency(adjency[:,:,0],6)).T

In [10]:
features = adjency[edges_index[0,],edges_index[1,],:]

In [196]:
def id2edges(id_,np_path, nodes):
    features_map = np.load(str(np_path/(id_+".npy")))
    edges = knn_from_adjency(features_map[:,:,0])
    edge_index = np.array(knn_from_adjency(features_map[:,:,0],6)).T
    edge_features = features_map[edge_index[0,],edge_index[1,],:]
    edge_type = np.stack([(edge_index[1,] - edge_index[0,] == 1).astype(int),
                          (edge_index[0,] - edge_index[1,] == 1).astype(int)]).T
    edge_from = nodes[edge_index[0,]]
    edge_to = nodes[edge_index[1,]]
    edge_features = np.concatenate([edge_features,edge_type, edge_from, edge_to],axis=-1)
    return edge_index, edge_features

In [12]:
df.columns

Index(['index', 'id', 'sequence', 'structure', 'predicted_loop_type',
       'signal_to_noise', 'SN_filter', 'seq_length', 'seq_scored',
       'reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10',
       'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 'deg_Mg_pH10',
       'deg_pH10', 'deg_Mg_50C', 'deg_50C'],
      dtype='object')

In [13]:
df.columns

Index(['index', 'id', 'sequence', 'structure', 'predicted_loop_type',
       'signal_to_noise', 'SN_filter', 'seq_length', 'seq_scored',
       'reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10',
       'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 'deg_Mg_pH10',
       'deg_pH10', 'deg_Mg_50C', 'deg_50C'],
      dtype='object')

In [5]:
prefix, suffix = 'deg_Mg_50C'.split("_",1)
prefix+"_error_"+suffix

'deg_error_Mg_50C'

In [90]:
def build_data(df, target_cols=None):
    target_cols = target_cols or []
    data_list = []
    for id_, sequence,seq_scored,  *targets in tqdm(df[['id','sequence','seq_scored']+target_cols].values):
        x = seq2nodes(sequence)
        edge_index, edge_features = id2edges(id_, Path(MAP2D_FOLDER), x)
        if targets:
            targets = torch.from_numpy(np.stack(targets).T)
        else:
            targets = None
        edge_index = torch.LongTensor(edge_index)
        edge_features = torch.FloatTensor(edge_features)
        x = torch.FloatTensor(x)
        data = Data(x, edge_index, edge_features, targets)
        data.seq_scored = seq_scored
        data_list.append(data)
    assert len({data.seq_scored for data in data_list}) == 1
    return data_list

In [178]:
def seq2nodes_looptype(looptype):
    vocab = ["S", "M", "I", "B", "H", "E", "X"]
    type_dict={s:i for i,s in enumerate(vocab)}
    nodes=np.array([type_dict[lp] for lp in looptype])
    return nodes

In [162]:
def build_data_looptype(df):
    data_list = []
    for id_, sequence,seq_scored, looptype,  *targets in tqdm(df[['id',
                                                                'sequence',
                                                                'seq_scored','predicted_loop_type']].values):
        x = seq2nodes(sequence)
        edge_index, edge_features = id2edges(id_, Path(MAP2D_FOLDER), x)
        targets = torch.from_numpy(seq2nodes_looptype(looptype))
        edge_index = torch.LongTensor(edge_index)
        edge_features = torch.FloatTensor(edge_features)
        x = torch.FloatTensor(x)
        data = Data(x, edge_index, edge_features, targets)
        data.seq_scored = seq_scored
        data_list.append(data)
    assert len({data.seq_scored for data in data_list}) == 1
    return data_list

In [15]:
np_files = []
for file in Path('../data/nsp_distances_angles2/').glob("*.npy"):
    np_files.append(file.stem)
np_files = set(np_files)

In [16]:
df = df[df.id.apply(lambda a: a in np_files)]

In [65]:
class MyDataModule(pl.LightningDataModule):

    def __init__(self, df_train, df_val, batch_size):
        super().__init__()
        self.batch_size = batch_size
        self.df_train = df_train
        self.df_val = df_val
    
    def setup(self, stage=None):
        self.train_data_list = build_data(self.df_train, target_cols=TARGET_COLS)
        self.val_data_list = build_data(self.df_val, target_cols=TARGET_COLS)
        
    def train_dataloader(self):
        return DataLoader(self.train_data_list, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_data_list, batch_size=self.batch_size)

In [175]:
class MyDataModuleLT(MyDataModule):
    def setup(self, stage=None):
        self.train_data_list = build_data_looptype(self.df_train)
        self.val_data_list = build_data_looptype(self.df_val)

In [66]:
def compute_MCRMSE(pred, y, columns, seq_len, seq_scored, per_column=False,prefix=''):
    pred = pred.view(-1, seq_len, pred.shape[-1])
    pred = pred[:,:seq_scored,:].reshape(-1, pred.shape[-1])
    losses = torch.sqrt(torch.mean((pred - y)**2,dim=0) + 1e-6)
    if not per_column:
        return losses.mean()
    metrics = {f"{prefix}_mcrmse_{col}" if prefix else f"mcrmse_{col}":loss for col,loss in zip(columns, losses)}
    metrics["mcrmse"] = losses.mean()
    return metrics
    

In [169]:
class MyGNNLighting(pl.LightningModule):
    
    def __init__(self,input_size, hidden_size, edge_dim, output_dim=3, seq_len=107, seq_scored=68):
        super().__init__()
        self.seq_len = seq_len
        self.lstm = nn.LSTM(input_size,int(hidden_size/2),bidirectional=True, batch_first=True)
        self.gcn = gnn.CGConv(hidden_size, edge_dim)
        self.gcn2 = gnn.CGConv(hidden_size, edge_dim)
        self.mlp = nn.Linear(hidden_size, output_dim)
        self.seq_scored = seq_scored
    
    def forward(self, data):
        x = data.x.float()
#         print(x)
        x = self.lstm(x.view(-1, self.seq_len, x.shape[-1]))[0]
#         print(x.shape)
        x = x.reshape(-1,x.shape[-1])
        x = self.gcn(x,data.edge_index, data.edge_attr)
        x = torch.sigmoid(x)
        x = self.gcn2(x,data.edge_index, data.edge_attr)
        x = self.mlp(x)
        return x
    
    def training_step(self, batch, batch_idx):
        pred = self(batch)
        y = batch.y
        return self.compute_loss(pred, y)
    
    def validation_step(self, batch, batch_idx):
        pred = self(batch)
        y = batch.y
        return (pred, y)
    
    def validation_epoch_end(self, validation_step_outputs):
        pred,y = zip(*validation_step_outputs)
        pred = torch.cat(pred,dim=0)
        y = torch.cat(y, dim=0)
        metrics = self.compute_loss(pred, y, per_column=True, prefix='val')
        result = pl.EvalResult(checkpoint_on=metrics['mcrmse'])
        result.log_dict(metrics, on_step=False, on_epoch=True, logger=True, prog_bar=False)
        return result
    
    def compute_loss(self, pred, y,columns=TARGET_COLS, per_column=False, prefix=''):
        return compute_MCRMSE(pred, y, columns, self.seq_len, self.seq_scored, per_column, prefix=prefix)
    
    def configure_optimizers(self):
        opt = Adam(self.parameters(), lr=1e-2)
        return opt
        

In [191]:
class MyGNNLightingLT(pl.LightningModule):
    
    
    def __init__(self,input_size, hidden_size, edge_dim, output_dim=7, seq_len=107, seq_scored=107):
        super().__init__()
        self.seq_len = seq_len
        self.lstm = nn.LSTM(input_size,int(hidden_size/2),bidirectional=True, batch_first=True)
        self.gcn = gnn.CGConv(hidden_size, edge_dim)
        self.gcn2 = gnn.CGConv(hidden_size, edge_dim)
        self.mlp = nn.Linear(hidden_size, output_dim)
        self.seq_scored = seq_scored
        self.loss = nn.CrossEntropyLoss()
    
    def forward(self, data):
        x = data.x.float()
#         print(x)
        x = self.lstm(x.view(-1, self.seq_len, x.shape[-1]))[0]
#         print(x.shape)
        x = x.reshape(-1,x.shape[-1])
        x = self.gcn(x,data.edge_index, data.edge_attr)
        x = torch.sigmoid(x)
        x = self.gcn2(x,data.edge_index, data.edge_attr)
        x = self.mlp(x)
        return x
    
    def training_step(self, batch, batch_idx):
        pred = self(batch)
        y = batch.y
        return self.compute_loss(pred, y)
    
    def validation_step(self, batch, batch_idx):
        pred = self(batch)
        y = batch.y
        return (pred, y)
    
    def validation_epoch_end(self, validation_step_outputs):
        pred,y = zip(*validation_step_outputs)
        pred = torch.cat(pred,dim=0)
        y = torch.cat(y, dim=0)
        loss = self.compute_loss(pred, y)
        result = pl.EvalResult(checkpoint_on=loss)
        result.log("val_loss", loss, on_step=False, on_epoch=True, logger=True, prog_bar=False)
        return result
    
    def compute_loss(self, pred, y):
        return self.loss(pred, y)
    
    def configure_optimizers(self):
        opt = Adam(self.parameters(), lr=1e-4)
        return opt
        

In [91]:
df_train = df.iloc[:1000]
df_test = df.iloc[1000:]

In [163]:
data = MyDataModule(df_test.iloc[:100],df_test.iloc[:100],16)

In [164]:
data.setup()

In [170]:
module = MyGNNLighting(4,100,12)

In [166]:
logger = pl.loggers.neptune.NeptuneLogger(NEPTUNE_API_TOKEN,"gottalottarock/openVaccine",experiment_name="test")

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/gottalottarock/openVaccine/e/OP-8


NeptuneLogger will work in online mode


In [172]:
trainer = pl.trainer.Trainer(logger=logger,max_epochs=40)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [173]:
trainer.fit(module,data)


  | Name | Type   | Params
--------------------------------
0 | lstm | LSTM   | 22 K  
1 | gcn  | CGConv | 42 K  
2 | gcn2 | CGConv | 42 K  
3 | mlp  | Linear | 303   


Failed to send channel value.
Traceback (most recent call last):
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/internal/backends/hosted_neptune_backend.py", line 560, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchErr

Failed to send channel value.
Traceback (most recent call last):
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/internal/backends/hosted_neptune_backend.py", line 560, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchErr

Failed to send channel value.
Traceback (most recent call last):
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/internal/backends/hosted_neptune_backend.py", line 560, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchErr

Failed to send channel value.
Traceback (most recent call last):
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/internal/backends/hosted_neptune_backend.py", line 560, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchErr

Failed to send channel value.
Traceback (most recent call last):
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/internal/backends/hosted_neptune_backend.py", line 560, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchErr

KeyboardInterrupt: 

Failed to send channel value.
Traceback (most recent call last):
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/internal/channels/channels_values_sender.py", line 156, in _send_values
    self._experiment._send_channels_values(channels_with_values)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/experiments.py", line 1138, in _send_channels_values
    self._backend.send_channels_values(self, channels_with_values)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/utils.py", line 210, in wrapper
    return func(*args, **kwargs)
  File "/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/neptune/internal/backends/hosted_neptune_backend.py", line 560, in send_channels_values
    raise ChannelsValuesSendBatchError(experiment.id, batch_errors)
neptune.api_exceptions.ChannelsValuesSendBatchErr

In [150]:
for batch in data.val_dataloader():
    break

In [204]:
data = MyDataModuleLT(df_train, df_test,16)

In [205]:
data.setup()

In [206]:
logger = pl.loggers.neptune.NeptuneLogger(NEPTUNE_API_TOKEN,"gottalottarock/openVaccine",experiment_name="test_LT")

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/gottalottarock/openVaccine/e/OP-13


NeptuneLogger will work in online mode


In [201]:
trainer = pl.trainer.Trainer(logger=logger,max_epochs=40)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [202]:
module = MyGNNLightingLT(4,100,12)

In [203]:
trainer.fit(module,data)


  | Name | Type             | Params
------------------------------------------
0 | lstm | LSTM             | 22 K  
1 | gcn  | CGConv           | 42 K  
2 | gcn2 | CGConv           | 42 K  
3 | mlp  | Linear           | 707   
4 | loss | CrossEntropyLoss | 0     
/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/media/stepan/files/projects/private/openVaccine/envs/graph/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Saving latest checkpoint..


1